3. ライブラリによる高度なクローリング・スクレイピング

  - 3-1. HTMLのスクレイピング
  - 3-2. XMLのスクレイピング
  - 3-3. データベースに保存する
  - 3-4. クローラーとURL
  - 3-5. Pythonによるクローラーの作成
  - 3-6. まとめ

## 3-1. HTMLのスクレイピング

- Beautiful Soup
  - APIでデータを抜き出せるのが特徴。
  - 目的に応じて、内部のパーサーを切り替えられる。
- pyquery
  - jQueryと同じようなインターフェイスでスクレイピングできる。
  - 内部でlxmlを使用している。

In [1]:
# bs4モジュールからBeautifulSoupクラスをインポートする。
from bs4 import BeautifulSoup

# 第1引数にファイルオブジェクトを指定して、BeautifulSoupオブジェクトを生成する。
# BeautifulSoup()にはファイル名やURLを指定することはできない。
# 第2引数にパーサー名を指定する。
with open('../ch02/dp.html') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
soup

<!DOCTYPE HTML>

<html class="pc" lang="ja">
<head>
<meta charset="utf-8"/>
<title>Gihyo Digital Publishing … 技術評論社の電子書籍</title>
<meta content="text/css" http-equiv="Content-Style-Type">
<meta content="application/javascript" http-equiv="Content-Script-Type"/>
<meta content="技術評論社の電子書籍（電子出版）販売サイト" name="description"/>
<meta content="電子書籍,電子出版,EPUB,PDF,技術評論社" name="keywords"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="telephone=no" name="format-detection"/>
<link href="http://gihyo.jp/dp/catalogs.opds" rel="related" title="Gihyo Digital Publishing OPDS Catalog" type="application/atom+xml;profile=opds-catalog"/>
<link href="/assets/templates/gdp/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<link href="/dp/assets/gdp-icon.png" rel="apple-touch-icon-precomposed"/>
<!--[if lt IE 9]>
    <script>var msie=8;</script>
    <script src="//ajax.googleapis.com/ajax/libs/jquery/1.11.

In [2]:
from bs4 import BeautifulSoup

# BeautifulSoupのコンストラクターにはHTMLの文字列を渡すこともできる。
soup = BeautifulSoup("""
    <html>
    <head><title>八百屋オンライン</title></head>
    <body>
    <h1 id=main><strong>おいしい</strong>今日のくだもの</h1>
    <ui>
        <li>りんご</li>
        <li class="featured">みかん</li>
        <li>ぶどう</li>
    </ui>
    </html>
    </body>
""", 'html.parser')

In [3]:
# soup.titleのようにタグ名の属性で、title要素を取得できる。
soup.title

<title>八百屋オンライン</title>

In [4]:
# 要素はTagオブジェクト
type(soup.title)

bs4.element.Tag

In [5]:
# Tagオブジェクトのname属性でタグ名を取得できる。
soup.title.name

'title'

In [6]:
# Tagオブジェクトのstring属性で要素の直接の子である文字列を取得できる。
soup.title.string

'八百屋オンライン'

In [7]:
# string属性で得られる文字列はstrを継承したNavigableStringオブジェクト。
type(soup.title.string)

bs4.element.NavigableString

In [8]:
# 着火の要素が文字列のみでない場合、string属性はNone。
soup.h1.string

In [9]:
# contents属性で子要素（NavigableStringを含む）のリストを取得できる。
soup.h1.contents

[<strong>おいしい</strong>, '今日のくだもの']

In [10]:
# text属性で要素内のすべての文字列を結合した文字列を取得できる。
soup.h1.text

'おいしい今日のくだもの'

In [12]:
# text属性で得られる文字列はstrオブジェクト。
type(soup.h1.text)

str

In [13]:
# Tagオブジェクトはdictのようにして属性を取得できる。
soup.h1['id']

'main'

In [14]:
# dictと同様にget()メソッドでも属性を取得できる。
soup.h1.get('id')

'main'

In [15]:
# attrs属性で全属性を表すdictオブジェクトを取得できる。
soup.h1.attrs

{'id': 'main'}

In [16]:
# parent属性で親要素（body要素）を取得できる。
soup.h1.parent

<body>
<h1 id="main"><strong>おいしい</strong>今日のくだもの</h1>
<ui>
<li>りんご</li>
<li class="featured">みかん</li>
<li>ぶどう</li>
</ui>
</body>

In [17]:
# 複数の要素がある場合は、先頭の要素が取得される。
soup.li

<li>りんご</li>

In [18]:
# find()メソッドも同様。
soup.find('li')

<li>りんご</li>

In [19]:
# find_all()メソッドで指定した名前の要素のリストを取得できる。
soup.find_all('li')

[<li>りんご</li>, <li class="featured">みかん</li>, <li>ぶどう</li>]

In [20]:
# キーワード引数でclassなどの属性を指定できる。
# classは予約語なので、class_を使う。
soup.find_all('li', class_="featured")

[<li class="featured">みかん</li>]

In [21]:
# タグ名を省略して、属性のみで探すことも可能。
soup.find_all(id='main')

[<h1 id="main"><strong>おいしい</strong>今日のくだもの</h1>]

In [22]:
# select()メソッドでCSSセレクターにマッチする要素を取得できる。
# BeautifulSoupでは、CSSセレクターのサブセットのみがサポートされている。
soup.select('li')

[<li>りんご</li>, <li class="featured">みかん</li>, <li>ぶどう</li>]

In [23]:
soup.select('li.featured')

[<li class="featured">みかん</li>]

In [24]:
soup.select('#main')

[<h1 id="main"><strong>おいしい</strong>今日のくだもの</h1>]

In [28]:
# scrape_by_bs4.py
# Beautiful Soup 4でスクレイピングする

from urllib.parse import urljoin
from bs4 import BeautifulSoup

# HTMLファイルを読み込んでBeautifulSoupオブジェクトを取得する。
with open('../ch02/dp.html') as f:
    soup = BeautifulSoup(f, 'html.parser')

# select()メソッドで、セレクターに該当するa要素のリストを取得して、
# 個々のa要素に対して処理を行う。
for a in soup.select('#listBook > li > a[itemprop="url"]'):
    # a要素のhref属性から書籍のURLを取得する。
    url = urljoin('https://gihyou.jp/dp', a.get('href'))
    
    # 書籍タイトルは、itemprop="name"という属性を持つp要素から取得する。
    p = a.select('p[itemprop="name"]')[0]
    # wbr要素などが含まれるのでstringではなくtextを使う。
    title = p.text
    
    # 書籍のURLとタイトルを出力する。
    print(url, title)

https://gihyou.jp/dp/ebook/2023/978-4-297-13486-0 今すぐ使えるかんたん 今すぐ使えるかんたんぜったいデキます！ パワーポイント超入門［Office 2021／Microsoft 365両対応］
https://gihyou.jp/dp/ebook/2023/978-4-297-13504-1 ゼロからはじめる ゼロからはじめるドコモ arrows N F-51C スマートガイド
https://gihyou.jp/dp/ebook/2023/978-4-297-13462-4 知りたい！サイエンス ゼータへの最初の一歩 ベルヌーイ数～「べき乗和」と素数で割った「余り」の驚くべき関係～
https://gihyou.jp/dp/ebook/2023/978-4-297-13442-6 今すぐ使えるかんたん 今すぐ使えるかんたんiPad完全ガイドブック 困った解決&便利技［iPadOS 16対応版］
https://gihyou.jp/dp/ebook/2023/978-4-297-13490-7 売れるランディングページ改善の法則
https://gihyou.jp/dp/ebook/2023/978-4-297-13241-5 ディープラーニングG検定（ジェネラリスト） 法律・倫理テキスト
https://gihyou.jp/dp/ebook/2023/978-4-297-13496-9 エンジニア選書 実践 Svelte入門
https://gihyou.jp/dp/ebook/2023/978-4-297-13510-2 現場が動きだす大学教育のマネジメントとは―茨城大学「教育の質保証」システム構築の物語
https://gihyou.jp/dp/ebook/2023/978-4-297-13508-9 Minecraftオフィシャルブック マインクラフト モブのたくらみ［石の剣のものがたりシリーズ②］
https://gihyou.jp/dp/ebook/2023/978-4-297-13408-2 図解即戦力 図解即戦力脱炭素のビジネス戦略と技術がこれ1冊でしっかりわかる教科書
https://gihyou.jp/dp/ebook/2023/978-4-297-13436-5 ノンプログラマーのための Vis

#### 3-1-2. pyqueryによるスクレイピング

In [ ]:
# PyQueryクラスをpqという名前でインポート。
from pyquery import PyQuery as pq

# ファイルパスを指定してパースできる。
d = pq(filename='index.html')

# URLを指定してパースすることもできる。
d = pq(url='http://example.com/')

In [1]:
from pyquery import PyQuery as pq

# 文字列を指定してパースすることもできる。
d= pq("""
    <html>
    <head><title>八百屋オンライン</title></head>
    <body>
    <h1 id=main><strong>おいしい</strong>今日のくだもの</h1>
    <ui>
        <li>りんご</li>
        <li class="featured">みかん</li>
        <li>ぶどう</li>
    </ui>
    </html>
    </body>
""")

In [2]:
# dはjQueryの$関数（jQuery関数）とほぼ同じ感覚で扱える。
# CSSセレクターを指定して、HTML要素に対応するオブジェクトを取得する。
d('h1')

[<h1#main>]

In [3]:
# 取得できるのは、listを継承したPyQueryクラスのオブジェクト。
# jQueryオブジェクトとほぼ同じ感覚で扱える。
type(d('h1'))

pyquery.pyquery.PyQuery

In [4]:
# リストの中身はpyqueryが内部的に使用しているlxmlのElement
d('h1')[0]

<Element h1 at 0x105544e00>

In [5]:
# text()メソッドで要素のテキストを取得できる。
d('h1').text()

'おいしい今日のくだもの'

In [8]:
# attr()メソッドで要素の属性を取得できる。
d('h1').attr('id')

'main'

In [10]:
# 属性やキーでも要素の属性にアクセスできる。
d('h1').attr.id

'main'

In [11]:
d('h1').attr['id']

'main'

In [12]:
# children()メソッドで子要素のリストを取得できる。
d('h1').children()

[<strong>]

In [13]:
# parent()メソッドで親要素のPyQueryオブジェクトを取得できる。
d('h1').parent()

[<body>]

In [15]:
# さまざまなCSSセレクターにマッチする要素のリストを取得できる。
d('li')

[<li>, <li.featured>, <li>]

In [16]:
d('#main')

[<h1#main>]

In [17]:
# jQueryのようにメソッドチェインで絞り込んでいくことも可能。
# find()メソッドで、現在の要素の子孫の中からCSSセレクターにマッチする要素のリストを取得できる。
d('body').find('li')

[<li>, <li.featured>, <li>]

In [18]:
# filter()メソッドで、現在の要素のリストの中からCSSセレクターにマッチする要素を絞り込める。
d('li').filter('.featured')

[<li.featured>]

In [19]:
# eq()メソッドで、現在の要素のリストの中から指定したインデックスの要素を取得できる。
d('li').eq(1)

[<li.featured>]

In [20]:
# scrape_by_pyquery.py
# pyqueryでスクレイピングする。

from pyquery import PyQuery as pq

# HTMLファイルを読み込んでPyQueryオブジェクトを取得する。
d = pq(filename='../ch02/dp.html')
# すべてのリンクを絶対URLに変換する。
d.make_links_absolute('http://gihyo.jp/dp')

# d()でセレクターに該当するa要素のリストを取得して、個々のa要素に対して処理を行う。
for a in d('#listBook > li > a[itemprop="url"]'):
    # a要素のhref属性から書籍のURLを取得する。
    # 変数aで取得できるのは、lxmlのElementなので、
    # d(a)としてPyQueryオブジェクトを取得している。
    url = d(a).attr('href')
    
    # 書籍のタイトルは、itemprop="name"という属性を持つp要素から取得する。
    p = d(a).find('p[itemprop="name"]').eq(0)
    title = p.text()
    
    print(url, title)

http://gihyo.jp/dp/ebook/2023/978-4-297-13486-0 今すぐ使えるかんたん 今すぐ使えるかんたん
ぜったいデキます！ パワーポイント超入門
［Office 2021
／
Microsoft 365
両対応］
http://gihyo.jp/dp/ebook/2023/978-4-297-13504-1 ゼロからはじめる ゼロからはじめる
ドコモ arrows N F-51C スマートガイド
http://gihyo.jp/dp/ebook/2023/978-4-297-13462-4 知りたい！サイエンス ゼータへの最初の一歩 ベルヌーイ数
～
「べき乗和」
と素数で割った
「余り」
の驚くべき関係～
http://gihyo.jp/dp/ebook/2023/978-4-297-13442-6 今すぐ使えるかんたん 今すぐ使えるかんたん
iPad
完全ガイドブック 困った解決&
便利技
［iPadOS 16
対応版］
http://gihyo.jp/dp/ebook/2023/978-4-297-13490-7 売れるランディングページ改善の法則
http://gihyo.jp/dp/ebook/2023/978-4-297-13241-5 ディープラーニング
G
検定
（ジェネラリスト） 法律・
倫理テキスト
http://gihyo.jp/dp/ebook/2023/978-4-297-13496-9 エンジニア選書 実践 Svelte
入門
http://gihyo.jp/dp/ebook/2023/978-4-297-13510-2 現場が動きだす大学教育のマネジメントとは
―茨城大学
「教育の質保証」
システム構築の物語
http://gihyo.jp/dp/ebook/2023/978-4-297-13508-9 Minecraft
オフィシャルブック
マインクラフト モブのたくらみ
［石の剣のものがたりシリーズ②］
http://gihyo.jp/dp/ebook/2023/978-4-297-13408-2 図解即戦力 図解即戦力
脱炭素のビジネス戦略と技術がこれ
1
冊でしっかりわかる教科書
http://gihyo.jp/dp/ebook/2023/978-4-297-13436-5 ノンプログラマーの

## 3-2. XMLのスクレイピング

### 3-2-1. lxmlによるスクレイピング

RSSをパースする

In [21]:
# scrape_rss_by_lxml.py
# lxmlでRSSをパースする。

import lxml.etree

# parse()関数でファイルを読み込んでElementTreeオブジェクトを取得する。
tree = lxml.etree.parse('rss2.xml')
# getroot()メソッドでXMLのルート要素（この例ではrss要素）に対応するElementオブジェクトを取得する。
root = tree.getroot()

# xpath()メソッドでxPathにマッチする要素のリストを取得する。
# channel/item は、channel要素の子要素であるitem要素を表す。
for item in root.xpath('channel/item'):
    # item要素内にあるtitle要素の文字列を取得する。
    title = item.xpath('title')[0].text
    # item要素内にあるlink要素の文字列を取得する。
    url = item.xpath('link')[0].text
    # URLとタイトルを表示する。
    print(url, title)

https://gihyo.jp/article/2023/05/daily-linux-230501?utm_source=feed Debian 12.0 “Bookworm”のリリースは6月10日に
https://gihyo.jp/article/2023/05/ossdb-various-news0093?utm_source=feed 第93回　MySQL Day Tokyo開催、PostgreSQLの透過型暗号化機能とPGECons成果報告会
https://gihyo.jp/article/2023/05/blender-basics-09?utm_source=feed Blenderの機能「ベベル」とメッシュの外観の表示方法についてみてみよう
https://gihyo.jp/article/2023/04/python-bc-aichi2?utm_source=feed 「Python Boot Camp」2023年6月3日に名古屋で開催
https://gihyo.jp/admin/clip/01/ubuntu-topics/202304/28?utm_source=feed Ubuntu 23.10 "M" の準備、Ubuntu 23.04 日本語Remixのリリース候補版、Intel版のUbuntu
https://gihyo.jp/article/2023/04/deep-learning-future-morpho?utm_source=feed ディープラーニングの未来：東大とMorphoが目指す次のAI社会
https://gihyo.jp/article/2023/04/linux-learning-from-zero-knowledge?utm_source=feed 新人・未経験にこそオススメ Linuxをはじめよう！
https://gihyo.jp/article/2023/04/daily-linux-230427?utm_source=feed Linux 6.3リリース、ハードウェアサポートの強化、Steam Deckコントローラサポートなど
https://gihyo.jp/article/2023/04/lightning-network-overview-of-mercari?utm_source=feed 

### 3-2-2. RSSのスクレイピング

In [22]:
import feedparser

# parse()関数にURLを指定してパースできる。
d = feedparser.parse('http://b.hatena.ne.jp/hotentry/it.rss')
# parse()関数には、ファイルパス・ファイルオブジェクト・XMLの文字列も指定できる。
# d = feedparser.parse('it.rss')

In [23]:
# parse()関数の戻り値はFeedParserDictオブジェクト
type(d)

feedparser.util.FeedParserDict

In [24]:
# フィードのバージョンを取得する（この場合はRSS1.0）。
d.version

'rss10'

In [25]:
# フィールドのタイトルを取得する。
d.feed.title

'はてなブックマーク - 人気エントリー - テクノロジー'

In [26]:
# 属性ではなく、dict形式でもアクセスできる。
d['feed']['title']

'はてなブックマーク - 人気エントリー - テクノロジー'

In [27]:
# フィールドのリンクを取得する。
d.feed.link

'https://b.hatena.ne.jp/hotentry/it'

In [28]:
# フィードの説明を取得する。
d.feed.description

'最近の人気エントリー'

In [29]:
# d.entriesでフィードの要素のlistを取得する。
len(d.entries)

30

In [30]:
# 要素のタイトルを取得する。
d.entries[0].title

'『WEB+DB PRESS』 休刊のお知らせ：WEB+DB PRESS'

In [31]:
# 要素のリンクを取得する。
d.entries[0].link

'https://gihyo.jp/magazine/wdpress/suspended'

In [32]:
# 要素の説明を取得する。
d.entries[0].description

'WEB+DB PRESSは，2023年8月発売のVol.136をもって隔月刊誌としては休刊させていただきます。物価上昇による製作費の高騰など諸般の事情により，今回の決定に至った次第です。 突然の休刊案内にてたいへん恐縮ではございますが，何卒ご理解を賜りますよう，お願い申し上げます。 22年以上の長きにわたり，絶大なご支援を...'

In [33]:
# 要素の更新日時を文字列で取得する。
d.entries[0].updated

'2023-05-01T02:45:45Z'

In [35]:
# 要素の更新日時をパースして、time.struct_timeオブジェクトを取得する。
d.entries[0].updated_parsed

time.struct_time(tm_year=2023, tm_mon=5, tm_mday=1, tm_hour=2, tm_min=45, tm_sec=45, tm_wday=0, tm_yday=121, tm_isdst=0)

In [36]:
# scrape_rss_by_feedparser.py
# feedparserでRSSをスクレイピングする。

import feedparser

# はてなブックマークの人気エントリー（「テクノロジー」カテゴリ）のRSSを読み込む。
d = feedparser.parse('http://b.hatena.ne.jp/hotentry/it.rss')

# すべての要素について処理を繰り返す。
for entry in d.entries:
    # URLとタイトルを出力する。
    print(entry.link, entry.title)

https://gihyo.jp/magazine/wdpress/suspended 『WEB+DB PRESS』 休刊のお知らせ：WEB+DB PRESS
https://anond.hatelabo.jp/20230501005512 RSSリーダーが廃れた理由を教えて！
https://diamond.jp/articles/-/322071 楽天市場はなぜ「アマゾンより使いにくい」のか、IT批評家が徹底検証
https://zenn.dev/raltos/books/e50ef289c7565c ドッカー入門
https://www.watch.impress.co.jp/docs/series/nishida/1497700.html スマホの「フリック入力」から日本が見えてくる【西田宗千佳のイマトミライ】
https://firststar-hateno.hatenablog.com/entry/2023/05/01/165729 一番星はてのとおしゃべりできるようになりました - 一番星はての開発ブログ
https://qiita.com/yuno_miyako/items/ce80002adf76bd321ad3 【2024年版】ChatGPT APIを社内利用する時に採用すべきアーキテクチャを考えた - Qiita
https://zenn.dev/gibjapan/articles/1d8dfb7520dabc データベースのロックの基礎からデッドロックまで
https://www.publickey1.jp/blog/23/terraformhashicorpterraform_cloud.html Terraformがノーコードに。HashiCorpが「Terraform Cloud ノーコードプロビジョニング」正式リリース
https://www.itmedia.co.jp/mobile/articles/2305/01/news048.html iCloudは無料でOK　iPhoneで写真100万枚を保存できる「共有アルバム」が便利すぎる
https://pc.watch.impress.co.jp/docs/column/yajiuma-mini-review/1495391.html 【やじうまミニレビュー】 iPhoneで紛失防止

## 3-3. データベースに保存する

### 3-3-1. SQLite3への保存

In [1]:
# save_sqlite3.pyの実行
%run save_sqlite3.py

(1, '上海', 24150000)
(2, 'カラチ', 23500000)
(3, '北京', 21516000)
(4, '天津', 14722100)
(5, 'イスタンブル', 14160467)


### 3-3-2. MySQLへのデータの保存

In [2]:
!mysql --version

mysql  Ver 8.0.32 for macos13.0 on x86_64 (Homebrew)


In [3]:
!which mysql

/usr/local/bin/mysql


In [4]:
# MySqlサーバーの起動
!mysql.server start

Starting MySQL
.. SUCCESS! 


In [5]:
# MySqlサーバーの状態確認
!mysql.server status

 SUCCESS! MySQL running (6719)


#### データベースとユーザーの作成

Terminalで実行

```bash
# rootユーザーとしてMySqlサーバーに接続する
mysql -u root

Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 9
Server version: 8.0.32 Homebrew

Copyright (c) 2000, 2023, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

# データベース名: scraping
# 文字コード: utf8mb4
CREATE DATABASE scraping DEFAULT CHARACTER SET utf8mb4;

Query OK, 1 row affected (0.00 sec)

# localhostから接続可能な
# ユーザー名: scraper
# パスワード: password
CREATE USER scraper@localhost IDENTIFIED BY 'password';

Query OK, 0 rows affected (0.01 sec)

# ユーザーscraperにデータベースscrapingを読み書き可能な権限を与える
GRANT ALL ON scraping.* TO scraper@localhost;

Query OK, 0 rows affected (0.00 sec)

# データベースとの接続終了
exit

Bye
```

- データベースとの接続終了
  - `exit` + [enter]
  - [ctrl] + [D]

#### PythonからMySQLに接続する

- mysqlclient
- MySQL Connector/Python
- PyMySQL
- Python Database API 2.0

In [7]:
! python save_mysql.py

(1, '上海', 24150000)
(2, 'カラチ', 23500000)
(3, '北京', 21516000)
(4, '天津', 14722100)
(5, 'イスタンブル', 14160467)


```bash
# mysqlコマンドを使って、保存したデータを確認する
mysql -u scraper -p scraping -e 'SELECT * FROM `cities`'

password
```

### 3-3-3. MongoDBへのデータの保存

- [Install MongoDB Community Edition on macOS](https://www.mongodb.com/docs/v6.0/tutorial/install-mongodb-on-os-x/)
- [MongoDBをMacOSにインストール](https://qiita.com/fztkm/items/827200b1a5efaa28521c)

```bash
# https://github.com/mongodb/homebrew-brew
# 公式の MongoDB ソフトウェア用のカスタムHomebrewタップを追加する
brew tap mongodb/brew
```

```bash
# MongoDBをインストールする
# MongoDB Community Server
# MongoDB Database Tools
# MongoDB Compass
brew install mongodb-community
```

In [9]:
!mongod --version

db version v6.0.5
Build Info: {
    "version": "6.0.5",
    "gitVersion": "c9a99c120371d4d4c52cbb15dac34a36ce8d3b1d",
    "modules": [],
    "allocator": "system",
    "environment": {
        "distarch": "x86_64",
        "target_arch": "x86_64"
    }
}


In [10]:
!which mongod

/usr/local/bin/mongod


In [34]:
!brew --prefix

/usr/local


||Intel Processor|Apple Silicon Processor|
|:-:|:-:|:-:|
|configuration file|`/usr/local/etc/mongod.conf`|`/opt/homebrew/etc/mongod.conf`|
|log directory|`/usr/local/var/log/mongodb`|`/opt/homebrew/var/log/mongodb`|
|data directory|`/usr/local/var/mongodb`|`/opt/homebrew/var/mongodb`|

MongoDB（＝mongodプロセス）をmacOS Serviceとして実行する場合

- run
  - `brew services start mongodb-community`
- stop
  - `brew services stop mongodb-community`


MongoDB（＝mongodプロセス）をバックグラウンドで実行する場合

- run
  - `mongod --config /usr/local/etc/mongod.conf --fork` (Intel processors)
  - `mongod --config /opt/homebrew/etc/mongod.conf --fork` (Apple Silicon processors)
- stop
  - バックグラウンドプロセスとして動作している「mongod」を停止させるには、次の方法で「mongod」に接続します。
  - 'mongosh'を起動し、必要に応じて'shutdown'コマンドを発行してください。

どちらの方法も、インストール時に作成されたmongod.confファイルを使用します。このファイルには、独自のMongoDB設定オプションも追加することができます。

<https://www.mongodb.com/docs/v6.0/tutorial/install-mongodb-on-os-x/#run-mongodb-community-edition>

MongoDBの実行の確認

- foreground
  - `brew services list`
- background
  - `ps aux | grep -v grep | grep mongod`

MongoDBに接続して使用する

- `mongosh`

In [41]:
# MongoDBをフォアグラウンドで実行する。
!brew services start mongodb-community

==> Successfully started `mongodb-community` (label: homebrew.mxcl.mongodb-commu


In [42]:
# MongoDBの実行を確認
!brew services list

Name              Status  User   File
mongodb-community started takeru ~/Library/LaunchAgents/homebrew.mxcl.mongodb-community.plist
mysql             none           
unbound           none           


In [43]:
# MOngoDBに接続して使用
!mongosh

]0;mongosh mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000Current Mongosh Log ID:	6458d053c4b5510ac6d7f942
Connecting to:		mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.8.2
Using MongoDB:		6.0.5
Using Mongosh:		1.8.2

For mongosh info see: https://docs.mongodb.com/mongodb-shell/


To help improve our products, anonymous usage data is collected and sent to MongoDB periodically (https://www.mongodb.com/legal/privacy-policy).
You can opt-out by running the disableTelemetry() command.

 ------
   The server generated these startup warnings when booting
   2023-05-08T19:25:12.130+09:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
   2023-05-08T19:25:12.130+09:00: Soft rlimits for open file descriptors too low
------

------
   Enable MongoDB's free cloud-based monitoring service, which will then receive and display
   metrics about your depl

In [44]:
# PyMongoのインポート
from pymongo import MongoClient

In [46]:
# ホスト名とポート番号を指定して接続する。
# `27017` は、デフォルト値なので省略可能。
# client = MongoClient('localhost', 27017)
client = MongoClient('mongodb://localhost:27017/')
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [47]:
# testデータベースを取得する。
# データベースが存在しない場合でも、書き込み時に自動で作成される。
# db = client.test
# 属性で表せない場合、キーでも取得可能。
db = client['test']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')

In [48]:
# データベースのpostsコレクションを取得する。
# コレクションが存在しない場合でも、書き込み時に自動生成される。
# collection = db.posts
# 属性で表せない場合、キーでも取得可能。
collection = db['posts']
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'posts')

In [49]:
# insert_one()メソッドで、Pythonのdictをコレクションに追加できる。
collection.insert_one(
    {'name': '東京スカイツリー', 'prefecture': '東京'}
)

In [50]:
# insert_many()メソッドで、複数のdictを一度にコレクションに追加できる。
collection.insert_many(
    [
        {'name': '東京ディズニーランド', 'prefecture': '千葉'},
        {'name': '東京ドーム', 'prefecture': '東京'},
    ]
)

In [51]:
# find()メソッドで、すべてのドキュメントを取得するためのCursorオブジェクトを取得できる。
# すべてのドキュメントには、_idフィールドが自動で付与される。
# その値は、ObjectIdと呼ばれる12バイトの識別子。
for spot in collection.find():
    print(spot)

{'_id': ObjectId('6458d07b045ad7788ba58c6b'), 'name': '東京スカイツリー', 'prefecture': '東京'}
{'_id': ObjectId('6458d07f045ad7788ba58c6c'), 'name': '東京ディズニーランド', 'prefecture': '千葉'}
{'_id': ObjectId('6458d07f045ad7788ba58c6d'), 'name': '東京ドーム', 'prefecture': '東京'}


In [52]:
# find()メソッドの引数にクエリを指定すると、そのクエリにマッチするドキュメントを取得できる。
# 次のクエリは、prefectureフィールドの値が'東京'であるドキュメントを抽出する。
for spot in collection.find({'prefecture': '東京'}):
    print(spot)

{'_id': ObjectId('6458d07b045ad7788ba58c6b'), 'name': '東京スカイツリー', 'prefecture': '東京'}
{'_id': ObjectId('6458d07f045ad7788ba58c6d'), 'name': '東京ドーム', 'prefecture': '東京'}


In [53]:
# find_one()メソッドは、条件にマッチする最初のドキュメントを取得する。
collection.find_one()

{'_id': ObjectId('6458d07b045ad7788ba58c6b'),
 'name': '東京スカイツリー',
 'prefecture': '東京'}

In [54]:
# find()と同様に引数にクエリを指定できる。
collection.find_one({'prefecture': '千葉'})

{'_id': ObjectId('6458d07f045ad7788ba58c6c'),
 'name': '東京ディズニーランド',
 'prefecture': '千葉'}

In [55]:
# save_mongo.py 実行
!python save_mongo.py

6458d92bb5bb3a8fb9dafc64 https://gihyo.jp/dp/ebook/2023/978-4-297-13486-0 今すぐ使えるかんたん 今すぐ使えるかんたんぜったいデキます！ パワーポイント超入門［Office 2021／Microsoft 365両対応］
6458d92bb5bb3a8fb9dafc65 https://gihyo.jp/dp/ebook/2023/978-4-297-13504-1 ゼロからはじめる ゼロからはじめるドコモ arrows N F-51C スマートガイド
6458d92bb5bb3a8fb9dafc66 https://gihyo.jp/dp/ebook/2023/978-4-297-13462-4 知りたい！サイエンス ゼータへの最初の一歩 ベルヌーイ数～「べき乗和」と素数で割った「余り」の驚くべき関係～
6458d92bb5bb3a8fb9dafc67 https://gihyo.jp/dp/ebook/2023/978-4-297-13442-6 今すぐ使えるかんたん 今すぐ使えるかんたんiPad完全ガイドブック 困った解決&便利技［iPadOS 16対応版］
6458d92bb5bb3a8fb9dafc68 https://gihyo.jp/dp/ebook/2023/978-4-297-13490-7 売れるランディングページ改善の法則
6458d92bb5bb3a8fb9dafc69 https://gihyo.jp/dp/ebook/2023/978-4-297-13241-5 ディープラーニングG検定（ジェネラリスト） 法律・倫理テキスト
6458d92bb5bb3a8fb9dafc6a https://gihyo.jp/dp/ebook/2023/978-4-297-13496-9 エンジニア選書 実践 Svelte入門
6458d92bb5bb3a8fb9dafc6b https://gihyo.jp/dp/ebook/2023/978-4-297-13510-2 現場が動きだす大学教育のマネジメントとは―茨城大学「教育の質保証」システム構築の物語
6458d92bb5bb3a8fb9dafc6c https://gihyo.jp/dp/ebook/2023/978-4-297-

In [59]:
# MongoDBの停止
!brew services stop mongodb-community

Stopping `mongodb-community`... (might take a while)
==> Successfully stopped `mongodb-community` (label: homebrew.mxcl.mongodb-commu


In [60]:
# MongoDBの実行を確認
!brew services list

Name              Status User File
mongodb-community none        
mysql             none        
unbound           none        


## 3-4. クローラーとURL

### 3-4-1. パーマリンクとリンク構造のパターン

- Permalink（パーマリンク）
  - 時間が経過しても対応するコンテンツが変わらないURL
  - <https://gihyou.jp/dp/ebook/2018/978-4-7741-9770-8>
  - permanent（不変の）
- 一覧・詳細パターン
  - 一覧ページ <https://gihyou.jp/dp/>
  - 詳細ページ <https://gihyou.jp/dp/ebook/2018/978-4-7741-9770-8>


- Ajax
- pjax

### 3-4-2. 再実行を考慮したデータ設計

- データを一意に識別するキー
  - Yahoo!ファイナンスの株価情報
    - <https://stocks.finance.yahoo.co.jp/stocks/detail/?code=8411>
    - 証券コード
  - Amazon.co.jpの商品情報
    - <https://www.amazon.co.jp/dp/B00QJDQM9U>
    - ASIN
  - Twitterのツイート
    - <https://twitter.com/TwitterJP/status/1017144330211880960>
    - ツイートID
  - ITmediaニュースの記事
    - <https://www.itmedia.co.jp/news/articles/1807/26/news115.html>
    - 年月日と番号
- データベースの設計
  - サロゲートキーをプライマリーキーにする
  - MYSQL: AUTO_INCREMENT
  - MongoDB: ObjectId
  - uuidモジュール: UUID

## 3.5 Pythonによるっクローラーの作成

- Requests
  - Webページを取得
- lxml
  - Webページからスクレイピング
- PyMongo
  - MongoDBにデータを保存

&nbsp;

- 電子書籍販売サイト
  - 一覧ページ
    - タイトル
    - 価格
    - 著者
    - 書影
    - 対応フォーマット
    - 発売日
  - 詳細ページ
    - 対応する紙の書籍
    - 概要
    - 目次
    - サポート情報

### 3-5-1. 一覧ページからパーマリンク一覧を抜き出す

In [61]:
# 一覧ページからパーマリンク一覧を抜き出す (1)
!python python_crawler_1.py

https://gihyo.jp/dp/ebook/2023/978-4-297-13418-1
https://gihyo.jp/dp/ebook/2023/978-4-297-13456-3
https://gihyo.jp/dp/ebook/2023/978-4-297-13482-2
https://gihyo.jp/dp/ebook/2023/978-4-297-13486-0
https://gihyo.jp/dp/ebook/2023/978-4-297-13504-1
https://gihyo.jp/dp/ebook/2023/978-4-297-13462-4
https://gihyo.jp/dp/ebook/2023/978-4-297-13442-6
https://gihyo.jp/dp/ebook/2023/978-4-297-13490-7
https://gihyo.jp/dp/ebook/2023/978-4-297-13241-5
https://gihyo.jp/dp/ebook/2023/978-4-297-13496-9
https://gihyo.jp/dp/ebook/2023/978-4-297-13510-2
https://gihyo.jp/dp/ebook/2023/978-4-297-13508-9
https://gihyo.jp/dp/ebook/2023/978-4-297-13408-2
https://gihyo.jp/dp/ebook/2023/978-4-297-13436-5
https://gihyo.jp/dp/ebook/2023/978-4-297-13438-9
https://gihyo.jp/dp/ebook/2023/978-4-297-13478-5
https://gihyo.jp/dp/ebook/2023/978-4-297-13432-7
https://gihyo.jp/dp/ebook/2023/978-4-297-13434-1
https://gihyo.jp/dp/ebook/2023/978-4-297-13471-6
https://gihyo.jp/dp/ebook/2023/978-4-297-13414-3
https://gihyo.jp/dp/

In [2]:
# 一覧ページからパーマリンク一覧を抜き出す (2)
!python python_crawler_2.py

https://gihyo.jp/dp/ebook/2023/978-4-297-13418-1
https://gihyo.jp/dp/ebook/2023/978-4-297-13456-3
https://gihyo.jp/dp/ebook/2023/978-4-297-13482-2
https://gihyo.jp/dp/ebook/2023/978-4-297-13486-0
https://gihyo.jp/dp/ebook/2023/978-4-297-13504-1
https://gihyo.jp/dp/ebook/2023/978-4-297-13462-4
https://gihyo.jp/dp/ebook/2023/978-4-297-13442-6
https://gihyo.jp/dp/ebook/2023/978-4-297-13490-7
https://gihyo.jp/dp/ebook/2023/978-4-297-13241-5
https://gihyo.jp/dp/ebook/2023/978-4-297-13496-9
https://gihyo.jp/dp/ebook/2023/978-4-297-13510-2
https://gihyo.jp/dp/ebook/2023/978-4-297-13508-9
https://gihyo.jp/dp/ebook/2023/978-4-297-13408-2
https://gihyo.jp/dp/ebook/2023/978-4-297-13436-5
https://gihyo.jp/dp/ebook/2023/978-4-297-13438-9
https://gihyo.jp/dp/ebook/2023/978-4-297-13478-5
https://gihyo.jp/dp/ebook/2023/978-4-297-13432-7
https://gihyo.jp/dp/ebook/2023/978-4-297-13434-1
https://gihyo.jp/dp/ebook/2023/978-4-297-13471-6
https://gihyo.jp/dp/ebook/2023/978-4-297-13414-3
https://gihyo.jp/dp/

### 3-5-2. 詳細ページからスクレイピングする

|取得したい要素|CSSセレクター|
|:-:|:-:|
|タイトル|#bookTitle|
|価格|.by（直接の子要素である文字列のみ）|
|目次（章レベル）|#content > h3|

In [3]:
# 一覧ページからパーマリンク一覧を抜き出す (3)
# 最初の1ページのみ
!python python_crawler_3.py

{'url': 'https://gihyo.jp/dp/ebook/2023/978-4-297-13418-1', 'title': '因果推論入門〜ミックステープ：基礎から現代的アプローチまで', 'price': '3,800円 ', 'content': ['第1章\u3000導入', '第2章\u3000確率と回帰の概要', '第3章\u3000非巡回的有向グラフ', '第4章\u3000潜在アウトカム因果モデル', '第5章\u3000マッチングと層別化', '第6章\u3000回帰不連続デザイン', '第7章\u3000操作変数', '第8章\u3000パネルデータ', '第9章\u3000差分の差デザイン', '第10章\u3000合成コントロール法', '第11章\u3000結論']}


In [4]:
# 一覧ページからパーマリンク一覧を抜き出す (4)
# 最初の1ページのみ
# 価格の末尾の空白を削除する
!python python_crawler_4.py

{'url': 'https://gihyo.jp/dp/ebook/2023/978-4-297-13418-1', 'title': '因果推論入門〜ミックステープ：基礎から現代的アプローチまで', 'price': '3,800円', 'content': ['第1章\u3000導入', '第2章\u3000確率と回帰の概要', '第3章\u3000非巡回的有向グラフ', '第4章\u3000潜在アウトカム因果モデル', '第5章\u3000マッチングと層別化', '第6章\u3000回帰不連続デザイン', '第7章\u3000操作変数', '第8章\u3000パネルデータ', '第9章\u3000差分の差デザイン', '第10章\u3000合成コントロール法', '第11章\u3000結論']}


### 3-5-3. 詳細ページのクロールする

In [5]:
# 詳細ページをクロールする（1）
!python python_crawler_5.py

{'url': 'https://gihyo.jp/dp/ebook/2023/978-4-297-13418-1', 'title': '因果推論入門〜ミックステープ：基礎から現代的アプローチまで', 'price': '3,800円', 'content': ['第1章\u3000導入', '第2章\u3000確率と回帰の概要', '第3章\u3000非巡回的有向グラフ', '第4章\u3000潜在アウトカム因果モデル', '第5章\u3000マッチングと層別化', '第6章\u3000回帰不連続デザイン', '第7章\u3000操作変数', '第8章\u3000パネルデータ', '第9章\u3000差分の差デザイン', '第10章\u3000合成コントロール法', '第11章\u3000結論']}
{'url': 'https://gihyo.jp/dp/ebook/2023/978-4-297-13456-3', 'title': 'らくらく突破第7版 貸金業務取扱主任者 ○×問題＋過去問題集', 'price': '2,300円', 'content': ['第1章\u3000貸金業法および関係法令', '第2章\u3000貸付けに関する法令と実務', '第3章\u3000資金需要者等の保護', '第4章\u3000財務および会計']}
{'url': 'https://gihyo.jp/dp/ebook/2023/978-4-297-13482-2', 'title': 'デザインの学校これからはじめる Illustrator & Photoshopの本［2023年最新版］', 'price': '2,400円', 'content': ['Chapter 0\u3000IllustratorとPhotoshopの初期設定と基本操作', 'Chapter 1\u3000基本図形でかんたんな絵を描こう～Illustrator', 'Chapter 2\u3000下絵をなぞってイラストを描こう～Illustrator', 'Chapter 3\u3000イラストに色を塗ろう～Illustrator', 'Chapter 4\u3000ロゴタイプを作ろう～Illustrator', 'Chapter 5\u3000ロゴタイプに添えるマークを作ろう～Illust

In [6]:
# 詳細ページをクロールする（2）
# 正規表現で連続する空白を1つのスペースに置き換える。
!python python_crawler_6.py

{'url': 'https://gihyo.jp/dp/ebook/2023/978-4-297-13418-1', 'title': '因果推論入門〜ミックステープ：基礎から現代的アプローチまで', 'price': '3,800円', 'content': ['第1章 導入', '第2章 確率と回帰の概要', '第3章 非巡回的有向グラフ', '第4章 潜在アウトカム因果モデル', '第5章 マッチングと層別化', '第6章 回帰不連続デザイン', '第7章 操作変数', '第8章 パネルデータ', '第9章 差分の差デザイン', '第10章 合成コントロール法', '第11章 結論']}
{'url': 'https://gihyo.jp/dp/ebook/2023/978-4-297-13456-3', 'title': 'らくらく突破第7版 貸金業務取扱主任者 ○×問題＋過去問題集', 'price': '2,300円', 'content': ['第1章 貸金業法および関係法令', '第2章 貸付けに関する法令と実務', '第3章 資金需要者等の保護', '第4章 財務および会計']}
{'url': 'https://gihyo.jp/dp/ebook/2023/978-4-297-13482-2', 'title': 'デザインの学校これからはじめる Illustrator & Photoshopの本［2023年最新版］', 'price': '2,400円', 'content': ['Chapter 0 IllustratorとPhotoshopの初期設定と基本操作', 'Chapter 1 基本図形でかんたんな絵を描こう～Illustrator', 'Chapter 2 下絵をなぞってイラストを描こう～Illustrator', 'Chapter 3 イラストに色を塗ろう～Illustrator', 'Chapter 4 ロゴタイプを作ろう～Illustrator', 'Chapter 5 ロゴタイプに添えるマークを作ろう～Illustrator', 'Chapter 6 写真を編集しよう～Photoshop', 'Chapter 7 写真を加工しよう～Photoshop', 'Chapter 8 写真を切り抜いて合成しよう～Photosho

### 3-5-4. スクレイピングしたデータを保存する

In [1]:
# MongoDBをフォアグラウンドで実行する。
!brew services start mongodb-community

==> Successfully started `mongodb-community` (label: homebrew.mxcl.mongodb-commu


In [2]:
# MongoDBの実行を確認
!brew services list

Name              Status  User   File
mongodb-community started takeru ~/Library/LaunchAgents/homebrew.mxcl.mongodb-community.plist
mysql             none           
unbound           none           


In [4]:
# スクレイピングしたデータを保存する
!python python_crawler_final.py

Traceback (most recent call last):
  File "/Users/takeru/@LEARNING/Python/python_crawling_and_scraping_gihyo/ch03/python_crawler_final.py", line 111, in <module>
    main()
  File "/Users/takeru/@LEARNING/Python/python_crawling_and_scraping_gihyo/ch03/python_crawler_final.py", line 47, in main
    collection.insert_one(ebook)
  File "/Users/takeru/@LEARNING/Python/python_crawling_and_scraping_gihyo/venv/lib/python3.10/site-packages/pymongo/collection.py", line 628, in insert_one
    self._insert_one(
  File "/Users/takeru/@LEARNING/Python/python_crawling_and_scraping_gihyo/venv/lib/python3.10/site-packages/pymongo/collection.py", line 569, in _insert_one
    self.__database.client._retryable_write(acknowledged, _insert_command, session)
  File "/Users/takeru/@LEARNING/Python/python_crawling_and_scraping_gihyo/venv/lib/python3.10/site-packages/pymongo/mongo_client.py", line 1476, in _retryable_write
    return self._retry_with_session(retryable, func, s, None)
  File "/Users/takeru/@LEA

chatGPT のアドバイス

エラーメッセージは、コレクション "scraping.ebooks" に重複したキーエラーがあることを示します。具体的には、"key "フィールドに対してユニークインデックスが作成されており、ユニークな値を保持しているはずです。しかし、"key "フィールドに重複した値が存在し、それがエラーにつながったようです。これを解決するには、「key」フィールドがユニークな値を保持するようにコードを修正する必要があります。そのための一つの方法として、新しいドキュメントを挿入する前に、同じ「key」値を持つドキュメントがすでにコレクションに存在するかどうかをチェックします。

この修正では、新しいドキュメントを挿入する前に、同じ「キー」値を持つドキュメントがコレクションに既に存在するかどうかをチェックします。同じ "key "値を持つドキュメントがすでに存在する場合、コードは何もせずにこのケースを処理します。

In [5]:
# スクレイピングしたデータを保存する
!python python_crawler_final_2.py

{'url': 'https://gihyo.jp/dp/ebook/2023/978-4-297-13418-1', 'title': '因果推論入門〜ミックステープ：基礎から現代的アプローチまで', 'price': '3,800円', 'content': ['第1章 導入', '第2章 確率と回帰の概要', '第3章 非巡回的有向グラフ', '第4章 潜在アウトカム因果モデル', '第5章 マッチングと層別化', '第6章 回帰不連続デザイン', '第7章 操作変数', '第8章 パネルデータ', '第9章 差分の差デザイン', '第10章 合成コントロール法', '第11章 結論'], '_id': ObjectId('645cc9e4f7475cb15c7611a9')}
{'url': 'https://gihyo.jp/dp/ebook/2023/978-4-297-13456-3', 'title': 'らくらく突破第7版 貸金業務取扱主任者 ○×問題＋過去問題集', 'price': '2,300円', 'content': ['第1章 貸金業法および関係法令', '第2章 貸付けに関する法令と実務', '第3章 資金需要者等の保護', '第4章 財務および会計'], '_id': ObjectId('645cc9e5f7475cb15c7611aa')}
{'url': 'https://gihyo.jp/dp/ebook/2023/978-4-297-13482-2', 'title': 'デザインの学校これからはじめる Illustrator & Photoshopの本［2023年最新版］', 'price': '2,400円', 'content': ['Chapter 0 IllustratorとPhotoshopの初期設定と基本操作', 'Chapter 1 基本図形でかんたんな絵を描こう～Illustrator', 'Chapter 2 下絵をなぞってイラストを描こう～Illustrator', 'Chapter 3 イラストに色を塗ろう～Illustrator', 'Chapter 4 ロゴタイプを作ろう～Illustrator', 'Chapter 5 ロゴタイプに添えるマークを作ろう～Illustrator', 'Chapte

```bash
mongosh

Current Mongosh Log ID: 645ccb2ebcd249bc30211e6a
Connecting to:          mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.8.2
Using MongoDB:          6.0.5
Using Mongosh:          1.8.2

For mongosh info see: https://docs.mongodb.com/mongodb-shell/

------
   The server generated these startup warnings when booting
   2023-05-11T19:51:10.430+09:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
   2023-05-11T19:51:10.430+09:00: Soft rlimits for open file descriptors too low
------

------
   Enable MongoDB's free cloud-based monitoring service, which will then receive and display
   metrics about your deployment (disk utilization, CPU, operation statistics, etc).
   
   The monitoring data will be available on a MongoDB website with a unique URL accessible to you
   and anyone you share the URL with. MongoDB may use this information to make product
   improvements and to suggest MongoDB products and deployment options to you.
   
   To enable free monitoring, run the following command: db.enableFreeMonitoring()
   To permanently disable this reminder, run the following command: db.disableFreeMonitoring()
------
```

```bash
show dbs

admin      40.00 KiB
config     72.00 KiB
local      72.00 KiB
scraping  148.00 KiB
test       40.00 KiB
```

```bash
use scraping
switched to db scraping
```

```bash
scraping> show collections

books
ebooks
```

```bash
scraping> db.ebooks.find({})

[
  {
    _id: ObjectId("645b8c54350e0f08f89a59d8"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13418-1',
    title: '因果推論入門〜ミックステープ：基礎から現代的アプローチまで',
    price: '3,800円',
    content: [
      '第1章 導入',
      '第2章 確率と回帰の概要',
      '第3章 非巡回的有向グラフ',
      '第4章 潜在アウトカム因果モデル',
      '第5章 マッチングと層別化',
      '第6章 回帰不連続デザイン',
      '第7章 操作変数',
      '第8章 パネルデータ',
      '第9章 差分の差デザイン',
      '第10章 合成コントロール法',
      '第11章 結論'
    ]
  }
]
```

[mongodbでコレクションごとのレコード数を表示](https://qiita.com/bitarx/items/979298eaca2154189577)

```bash
scraping> db.getCollectionNames().forEach(function(n){print(n + ':' + db[n].find().count())})

(node:4623) [MONGODB DRIVER] Warning: cursor.count is deprecated and will be removed in the next major version, please use `collection.estimatedDocumentCount` or `collection.countDocuments` instead
(Use `node --trace-warnings ...` to show where the warning was created)
books:30
ebooks:1
```

```bash
scraping> db.books.find({})

[
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc64"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13486-0',
    title: '今すぐ使えるかんたん 今すぐ使えるかんたんぜったいデキます！ パワーポイント超入門［Office 2021／Microsoft 365両対応］'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc65"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13504-1',
    title: 'ゼロからはじめる ゼロからはじめるドコモ arrows N F-51C スマートガイド'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc66"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13462-4',
    title: '知りたい！サイエンス ゼータへの最初の一歩 ベルヌーイ数～「べき乗和」と素数で割った「余り」の驚くべき関係～'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc67"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13442-6',
    title: '今すぐ使えるかんたん 今すぐ使えるかんたんiPad完全ガイドブック 困った解決&便利技［iPadOS 16対応版］'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc68"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13490-7',
    title: '売れるランディングページ改善の法則'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc69"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13241-5',
    title: 'ディープラーニングG検定（ジェネラリスト） 法律・倫理テキスト'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc6a"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13496-9',
    title: 'エンジニア選書 実践 Svelte入門'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc6b"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13510-2',
    title: '現場が動きだす大学教育のマネジメントとは―茨城大学「教育の質保証」システム構築の物語'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc6c"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13508-9',
    title: 'Minecraftオフィシャルブック マインクラフト モブのたくらみ［石の剣のものがたりシリーズ②］'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc6d"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13408-2',
    title: '図解即戦力 図解即戦力脱炭素のビジネス戦略と技術がこれ1冊でしっかりわかる教科書'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc6e"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13436-5',
    title: 'ノンプログラマーのための Visual Studio Code実践活用ガイド'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc6f"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13438-9',
    title: '理解するほどおもしろい！ パソコンのしくみがよくわかる本 改訂3版'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc70"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13478-5',
    title: 'WEB+DB PRESS Vol.134'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc71"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13432-7',
    title: 'Nikon ニコン NX Studio パーフェクトガイド'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc72"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13434-1',
    title: '今すぐ使えるかんたん 今すぐ使えるかんたんOutlook完全ガイドブック 困った解決&便利技［Office 2021/2019/2016/Microsoft 365対応版］'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc73"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13471-6',
    title: '新居浜高専PICマイコン学習キット Ver.3 完全ガイド'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc74"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13414-3',
    title: '［速習］ベイズの定理——「推論」に効く数学の力'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc75"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13410-5',
    title: 'WEB+DB PRESS plus 縁の下のUIデザイン──小さな工夫で大きな効果をもたらす実践TIPS＆テクニック'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc76"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13446-4',
    title: '今すぐ使えるかんたんmini 今すぐ使えるかんたんminiExcel関数の基本と便利がこれ1冊でわかる本［Office 2021/Microsoft 365両対応］'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc77"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13469-3',
    title: 'はじめてつくるWebアプリケーション〜Ruby on Railsでプログラミングへの第一歩を踏み出そう'
  }
]
Type "it" for more
scraping> it
[
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc78"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13387-0',
    title: 'ポケットリファレンス HTML&CSSポケットリファレンス［改訂3版］'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc79"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13353-5',
    title: '英語は10000時間でモノになる～ハードワークで挫折しない「日本語断ち」の実践法～'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc7a"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13514-0',
    title: 'まんがですっきりわかる フリーランスのためのインボイスと消費税'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc7b"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13480-8',
    title: 'ポケットリファレンス Vue.jsポケットリファレンス'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc7c"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13404-4',
    title: '自由時間サプリ 誰でも簡単！ 世界一の4：6メソッドでハマる 美味しいコーヒー'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc7d"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13450-1',
    title: 'Software Design 2023年5月号'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc7e"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13357-3',
    title: 'ゼロからよくわかる！ Arduinoで電子工作入門ガイド 改訂2版'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc7f"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13458-7',
    title: 'COMODO（COMODO ライフブック） おとなの浴衣、はじめます'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc80"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13424-2',
    title: 'ポケット介護 【ポケット介護】改訂新版 見てわかるリハビリ～活動と参加で「自立支援」につなげるコツ'
  },
  {
    _id: ObjectId("6458d92bb5bb3a8fb9dafc81"),
    url: 'https://gihyo.jp/dp/ebook/2023/978-4-297-13393-1',
    title: '最短時間でムダなく成果を上げる パソコン仕事術の教科書［改訂新版］'
  }
]
```

```bash
scraping> quit
```

In [6]:
# MongoDBの停止
!brew services stop mongodb-community

Stopping `mongodb-community`... (might take a while)
==> Successfully stopped `mongodb-community` (label: homebrew.mxcl.mongodb-commu
